In [1]:
# attempts to optimize portfolio using the Efficient Frontier

In [2]:
import pandas_datareader as web
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
# FAANG (FACEBOOK, AMAZON ... )
assets = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOG']

In [4]:
# assign weights to stocks
weights = np.array([0.2,0.2,0.2,0.2,0.2])

In [5]:
# get the portfolio starting date
stockStartDate = '2013-01-01' # the first four years for Facebook

In [6]:
# ending date
today = datetime.today().strftime('%Y-%m-%d')
today

'2021-10-14'

In [7]:
# create a dataframe to store the adj close of the stocks
df = pd.DataFrame()

# store the adj close into df
for stock in assets:
    df[stock] = web.DataReader(stock, data_source='yahoo', start = '2017-01-01', end = '2021-04-13')['Adj Close']

RemoteDataError: Unable to read URL: https://finance.yahoo.com/quote/FB/history?period1=1483243200&period2=1618369199&interval=1d&frequency=1d&filter=history
Response Text:
b'<!DOCTYPE html>\n  <html lang="en-us"><head>\n  <meta http-equiv="content-type" content="text/html; charset=UTF-8">\n      <meta charset="utf-8">\n      <title>Yahoo</title>\n      <meta name="viewport" content="width=device-width,initial-scale=1,minimal-ui">\n      <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n      <style>\n  html {\n      height: 100%;\n  }\n  body {\n      background: #fafafc url(https://s.yimg.com/nn/img/sad-panda-201402200631.png) 50% 50%;\n      background-size: cover;\n      height: 100%;\n      text-align: center;\n      font: 300 18px "helvetica neue", helvetica, verdana, tahoma, arial, sans-serif;\n  }\n  table {\n      height: 100%;\n      width: 100%;\n      table-layout: fixed;\n      border-collapse: collapse;\n      border-spacing: 0;\n      border: none;\n  }\n  

In [ ]:
df

In [ ]:
# visually show portfolio
title = 'Portfolio Adj.Close Price Histroy'

# get the stocks
my_stocks = df

# create
plt.figure(figsize=(12,8))
for c in my_stocks.columns.values:
    plt.plot(my_stocks[c], label= c)
    
plt.title(title)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Adj. Close USD($) ', fontsize=18)
plt.legend(my_stocks.columns.values, loc='upper left')
plt.show()

In [ ]:
# show the daily simple return
returns = df.pct_change() #  pct_change = (day_price - previous_day_price) / previous_day_price
returns

In [ ]:
returns.cov()

In [ ]:
# create the annualized covariance matrix
cov_matrix_annual = returns.cov() * 252
cov_matrix_annual

In [ ]:
# calculate the portfolio variance
port_variance = np.dot(weights.T , np.dot(cov_matrix_annual, weights))
port_variance

In [ ]:
# calculate the portfolio volatility aka std
port_volatility = np.sqrt(port_variance)
port_volatility

In [ ]:
# calculate the annual portfolio return
portfolioSimpleAnnualReturn = np.sum(returns.mean()*weights)*252
portfolioSimpleAnnualReturn

In [ ]:
# show the expected annual return, volatility(risk), varicance
percent_var = str( round(port_variance, 2) * 100)+ '%'
percent_vols = str( round(port_volatility, 2) *100)+ '%'
percent_ret = str( round(portfolioSimpleAnnualReturn, 2)*100) + "%"

print('expected annual return:'+ percent_ret)
print('annual volatility/risk:'+ percent_vols)
print('annual variance:'+ percent_var)

let's see if we can get higher ret with lower vols

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models 
from pypfopt import expected_returns

In [ ]:
# portfolio optimization!

# calculate expected returns and the annualised sample cov matrix of asset returns
mu = expected_returns.mean_historical_return(df)
S = risk_models.sample_cov(df)

In [ ]:
# optimaize for max sharp ratio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
print(ef.portfolio_performance(verbose=True))

In [ ]:
# get the discrete allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=15000)

allocation, leftover = da.lp_portfolio()
print('Discrete allocation:', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))